# Regressão Polinomial

### Descrição:
Nesse notebook iremos aperfeiçoar o modelo de Regressão Linear que tínhamos, fazendo apenas algumas simples modificações. Uma Regressão Polinomial é um "nickname" para uma Regressão cujo modelo resultante será um polinômio de grau n. Essa técnica nos permite realizar uma Regressão mais complexa, no sentido de que nossos modelos podem representar mais do que relações lineares com relação aos dados estudados.

Apesar do nome, a Regressão Polinomial ainda é, essencialmente, uma Regressão Linear (pois esse termo se refere aos parâmetros, e não à relação entre as Variáveis). O algoritmo é, no fundo, o mesmo: apenas iremos fazer um tratamento especial nos dados antes de aplicarmos o Gradiente Descendente. Esse tratamento consiste em realizar transformações não-lineares nos dados e adicionar essa transformação como um novo atributo.

<b> Obs.: </b> todas as matrizes/vetores utilizados na fundamentação teórica são consideradas como Vetores-Colunas. A implementação pode diferir um pouco dessa convenção.


## Bibliotecas e Configurações

In [ ]:
# -*- coding: utf-8 -*-
%matplotlib qt5

# Libraries
import numpy as np 
import matplotlib.pyplot as plt

## Modelo Linear

No novo modelo, teremos novos atributos (que serão os atributos transformados) e, para cada um, novos parâmetros $\theta$. Por fim, nosso modelo poderá ter uma quantidade qualquer de atributos:

$$ h(\theta) = \theta_{0}+\theta_{1}X_{1}+\theta_{1}X_{2}+\cdots+\theta_{n}X_{n} $$

Apesar dessa mudança de representação, nossa operação matricial ainda é válida e, portante, a função utilizada anteriormente funcionará para o caso geral de qualquer um destes modelos.

$$ h(\theta) = \begin{bmatrix} \theta_{0} \\ \theta_{1} \\ \vdots \\ \theta_{n} \end{bmatrix}^{T} \times \begin{bmatrix} 1 \\ X_{1} \\ \vdots \\ X_{n} \end{bmatrix} = \begin{bmatrix} \theta_{0} & \theta_{1} & \cdots & \theta_{n} \end{bmatrix} \times \begin{bmatrix} 1 \\ X_{1} \\ \vdots \\ X_{n} \end{bmatrix}  = \theta^{T}X $$

In [ ]:
# Definição da Função para o Modelo Linear
def h_theta(x, theta):
    ''' Apply the Linear Model for features X and parameters theta '''
    return np.dot(np.transpose(theta), x)

# Teste da Função: h([5;2]) = 5+2X
testX = np.array([[1,1,1],
                  [3,4,9],
                  [9,16,81]])

testTheta = np.array([[5],
                      [2],
                      [0.5]])

print("Prediction:", h_theta(testX, testTheta))

## Função de Custo

Na célula abaixo, definimos uma função que calcula e retorna o custo total das nossas predições, isto é, o cálculo do erro total do treinamento. A função que utilizaremos pode ser arbitrária (apenas nos permite ter uma melhor visualização do treinamento, mas não influencia o mesmo).

Nesse algoritmo, utilizaremos a Soma dos Resíduos Quadráticos (um dos seus muitos nomes):

$$ J(\theta) = \frac{1}{m} \sum (h(\theta) - y)^{2} = \frac{1}{m} \sum ( Erro )^{2} $$

In [ ]:
# Definição da Função de Erro
def errorFunction(errors):
    ''' Calculate the Least Square Error '''
    return (1 / np.size(errors)) * np.sum(errors ** 2) 

# Teste da Função
errors = np.array([5., 4., 4., 3., 2.])
print("Custo Total:", errorFunction(errors))

## Extração de Features

Na célula abaixo, definimos a função que será responsável por extrair o conjunto de <i>features</i> de todo o Dataset. Utilizaremos essa função pois, com ela, podemos aumentar a complexidade do nosso modelo ao criar novos atributos que sejam transformações não-lineares dos <i>features</i> do Dataset.

É importante notar que caso o Dataset já possua $n$ <i>features</i>, cada transformação será aplicada às $n$ <i>features</i> e incluida no conjunto, implicando num crescimento de atributos igual a $(Kn)$, onde $K$ representa o grau de complexidade do polinômio dessa transformação.

In [ ]:
# Definição da Função de Extração
def featureExtraction(data, n_examples, complexity=1):
    ''' Extracts the features from a dataset and apply polynomial transformations '''
    x = np.ones(n_examples)
    
    for i in range(0, complexity):
        x = np.vstack([x, np.transpose(data[:, 0:-1]) ** (i+1)])
    
    return x

# Teste da Função
dataExtract = np.array([[2, 3, 0],
                        [5, 6, 0],
                        [8, 9, 0]])

x_extract = featureExtraction(dataExtract, 3, 1)
print("Features with complexity 1:\n", x_extract)
print("")

x_extract = featureExtraction(dataExtract, 3, 3)
print("Features with complexity 3:\n", x_extract)

## Feature Scaling

Na célula abaixo criamos uma função para <b>normalizar</b> as nossas <i>features</i>. O objetivo, como mostrado, é evitar problemas de divergência e permitir um melhor treinamento por meio dos hiperparâmetros.

A fórmula que usaremos para o <i>Feature Scaling</i> será:

$$
    X = \frac{X - \bar{X}}{\sigma(X)}
$$

In [ ]:
# Definição da Função de Normalização
def normalizeData(data):
    ''' Apply Feature Scaling to the features set '''
    for i in range(1, np.size(data,0)):
        data[i,:] = (data[i,:]-np.mean(data[i,:])) / np.std(data[i,:])
    
    return data

# Teste da Função
x_norm = np.array([[1., 1., 1., 1., 1.],
                   [2., 9., 4., 7., 5.],
                   [-5.,-4.,-2.,-9.,-8.]])

x_norm = normalizeData(x_norm)
print("Features normalizados:\n",x_norm)

## Programa Principal (Regressão Linear)

No programa principal, iremos programar a Regressão Linear propriamente dita.
Dividimos o código em três partes:

### Part 1: Data Pre-Processing

Nesse trecho, iremos nos preocupar em carregar e organizar o dataset que utilizaremos no treinamento. É nesse momento, também, que iremos declarar e separar as variáveis que definem o Conjunto de Atributos, o Conjunto de Saída e os Parâmetros do Modelo, além dos Hiperparâmetros de Treinamento. Iremos seguir a convenção de considerar todos os exemplares como vetores-colunas. No entanto, o numpy não nos permite facilmente modificar essa informação para o Conjunto de Saída, e o mesmo continuará como vetor-linha (sem muito prejuízo). Iremos criar, também um vetor para armazenar o Histórico de Erros do treinamento (por motivos de visualização).

Iremos utilizar o dataset <i>data1.txt</i> localizado na pasta <i>datasets/</i>. Teremos as seguintes matrizes:

$$
    X = \begin{bmatrix} 1 & 1 & \cdots & 1 \\  X_{1}^{(1)} & X_{1}^{(2)} & \cdots & X_{1}^{(m)}  \end{bmatrix};\   \theta = \begin{bmatrix} \theta_{0} \\ \theta_{1}\end{bmatrix};\  Y = \begin{bmatrix} Y^{(1)} & Y^{(2)} & \cdots & Y^{(m)} \end{bmatrix}
$$

### Part 2: Linear Regression Training

Para cada época até a convergência (ou até atingir o limite máximo definido pelo Hiperparâmetro) iremos realizar o Treinamento da Regressão Linear. Os passos serão os seguintes:

1. Calculamos o vetor de predição "Y_pred", como resultado da predição do Modelo para os parâmetros daquela época;
2. Utilizando "Y_pred", calculamos os erros de acordo com o a matriz real "Y";
3. Concatenamos o Custo Total do erro calculado no Histórico de Erros;
4. Realizamos, para cada parâmetro, o Gradiente Descendente para estimar os novos valores dos parâmetros;
5. Imprimimos os resultados do treino a cada 500 épocas;
6. Verificamos uma possível convergência do treino, e paremos o mesmo caso seja verificado;

### Part 3: Data Plotting and Training Results

Ao fim do treinamento, iremos plotar duas figuras para avaliar o resultado final do nosso algoritmo. A <b>Figura 1</b> irá apenas exibir os atributos do Dataset. A <b>Figura 2</b> irá exibir a função estimada pelo nosso Modelo Linear, além do Histórico de Erros dado as épocas até convergência.

In [ ]:
# Main Function
if __name__=='__main__':
    
    ###############################
    # Part 1: Data Pre-Processing #
    ###############################
    # Loads the data
    data = np.loadtxt("datasets/data1.txt")
    
    n_examples = np.size(data,0)
    
    # Define the model parameters
    x = normalizeData(featureExtraction(data, n_examples, 5))    
    n_features = np.size(x,0)

    y = data[:, 1]
    theta = np.zeros([np.size(x, 0), 1])

    # Defines the hyperparameters and training measurements
    alfa = 0.01
    max_epochs = 500000
    
    error_hist = np.zeros([max_epochs])
    epsilon = 0.001
    
    ######################################
    # Part 2: Linear Regression Training #
    ######################################
    for epochs in range(max_epochs):
        # Calculate the error vector from the current Model
        y_pred = h_theta(x, theta)
        error = y_pred - y

        # Append new Least Square Error to History
        error_hist[epochs] = errorFunction(error)

        # Perform Gradient Descent
        for j in range(n_features):
            theta[j] = theta[j] - (alfa/n_examples) * np.sum(error * x[j,:])

        # Prints training status at each 100 epochs
        if(epochs % 500 == 0):
            print("###### Epoch", epochs, "######")
            print("Error:", error_hist[epochs])
            print("Thetas:\n", theta)
            print("")
        
        # Evaluate convergence and stops training if so
        if(abs(error_hist[epochs] - error_hist[epochs-50]) <= epsilon):
            print("Gradient Converged!!!\nStopping at epoch", epochs)
            print("###### Epoch", epochs, "######")
            print("Error:", error_hist[epochs])
            print("Thetas:\n", theta)
            print("")
            break
            
    #############################################
    # Part 3: Data Plotting and Training Result #
    #############################################
    # First Figure: Dataset plotting
    plt.figure(1)
    
    plt.title("Artificial Generated Data with Noise\n $f(x)=-13.15648 + 1.4928 * X$")
    plt.xlabel("X")
    plt.ylabel("f(X)")
    
    plt.grid()
    plt.plot(x[1,:], y, 'rx')
    
    plt.show()
    
    # Second Figure: Training results
    plt.figure(2)
    
    plt.subplot(1,2,1)
    plt.title("Artificial Generated Data with Noise\n $f(x)=-13.15648 + 1.4928 * X$")
    plt.xlabel("X")
    plt.ylabel("f(X)")
    
    plt.grid()
    plt.plot(x[1,:], y, 'rx', x[1,:], h_theta(x, theta)[0,:], 'k-')
    
    plt.subplot(1,2,2)
    plt.title("Error History")
    plt.xlabel("Epochs")
    plt.ylabel("Least Square Error")
    
    plt.grid()
    plt.plot(error_hist[:epochs], "g-")
    
    plt.show()
    
#__